# DL1 Assignment2 - Q1.1 draft code

This is a small help from us to save you some coding. This notebook is **not** graded, you are free to edit it.

Further advise:
1. Start with File/Save a copy in Drive
2. Set GPU usage under Runtime/Change runtime type/Hardware accelerator.

In [1]:
%pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 6.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch import nn
import timm
from torchvision import models
from matplotlib import pyplot as plt
from typing import Callable

/Users/Matey/miniconda3/envs/mlkit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def vit_s_8():
    """ViT-S/8 is not a default torchvision model, so we provide it by timm"""
    # Accuracy approximation comes from
    # https://openreview.net/pdf?id=LtKcMgGOeLt
    # and DINO
    # https://arxiv.org/abs/2104.14294
    return timm.create_model('vit_small_patch8_224')

# Model definitions
# Optional Q: These are uncalled functions. What do you think would happen
# if we called all of them once? Why didn't we do that?
model_defs = [
    vit_s_8,
    models.vit_b_32,
    models.vgg11,
    models.vgg11_bn,
    models.resnet18,
    models.densenet121,
    models.mobilenet_v3_small,
]

# Accuracies per model
model_accs = {
    'vit_s_8': 80., # Approximated
    'vit_b_32' : 75.912,
    'vgg11' : 69.02,
    'vgg11_bn' : 70.37,
    'resnet18' : 69.758,
    'densenet121' : 74.434,
    'mobilenet_v3_small' : 67.668,
}


def measure_runtime_per_forward(model:nn.Module, no_grad=None, batch_size:int=8,device='cuda'):
    """Measures the time for a single pass in milliseconds"""

    # Generate fake RGB input (224x224)
    #######################
    # PUT YOUR CODE HERE  #
    #######################
    inp = torch.randn((batch_size, 3, 224, 224)).to(device)
    #######################
    # END OF YOUR CODE    #
    #######################

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()

    # Run the model
    #######################
    # PUT YOUR CODE HERE  #
    #######################

    if no_grad == "no_grad":
      with torch.no_grad():
        model(inp)
    elif no_grad == "eval":
      with torch.evaluate():
        model(inp)
    else:
      model(inp)

    #######################
    # END OF YOUR CODE    #
    #######################

    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)


def evaluate_model(model_def:Callable, no_grad: str|None, batch_size:int=8, n_warmup=10, n_repeat=100, device='cuda'):

    # Retreive initial memory allocation
    initial_vram = torch.cuda.memory_allocated()

    # Define model
    model = model_def().to(device).eval()
    # Access name as: model.__name__

    # Parameters that need to be filled
    n_params = None
    times, vrams = [], []
    mean_time = None
    mean_vram = None

    #######################
    # PUT YOUR CODE HERE  #
    #######################

    # Step 1: Calculate the number of **trainable** parameters
    n_params = sum(e.numel() for e in model.parameters())

    # Step 2: Warm up with a few passes
    for _ in range(n_warmup):
      measure_runtime_per_forward(model, no_grad=no_grad,batch_size=batch_size,device=device)
    # Step 3: Run N forward passes and save the runtime +
    #         the vram allocated by the model

    for _ in range(n_repeat):
      time = measure_runtime_per_forward(model, no_grad=no_grad,batch_size=batch_size,device=device)
      times.append(time)
      vrams.append(torch.cuda.memory_allocated()/1e4)
    # Step 4: Take the mean, preferably with dropping possible outliers

    times = torch.tensor(times)
    vrams = torch.tensor(vrams).double()

    times = times[(times-times.mean())<=3*times.std()]
    vrams = vrams[(vrams-vrams.mean())<=3*vrams.std()]

    mean_time = times.mean().item()
    mean_vram = times.mean().item()
    #######################
    # END OF YOUR CODE    #
    #######################

    # Clean up space for the model
    del model
    torch.cuda.empty_cache()

    return mean_time, mean_vram, n_params


In [9]:
#######################
# PUT YOUR CODE HERE  #
#######################

# Example usage of the above functions:
df = []
for model_def in model_defs:
    name = model_def.__name__
    time, vram, n_params = evaluate_model(model_def, no_grad=True,device='mps')
    df.append((name, time, vram, n_params))

import pandas as pd
df = pd.DataFrame(df, columns=['name', 'mean_time (ms)', 'mean_vram (kb)', 'n_params'])
df['acc'] = list(model_accs.values())
display(df)
# Make your plots here with matplotlib
#
# plt.scatter()


#######################
# END OF YOUR CODE    #
#######################

RuntimeError: Tried to instantiate dummy base class Event

In [ ]:
torch.d